### **Library Imports**

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import random as r
import seaborn as sns
import matplotlib.pyplot as plt

from time import time
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, GridSearchCV

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, classification_report

### **Configuration**

In [2]:
class CFG(object):
    def __init__(self,
                 seed: int = 42,
                 n_splits: int = 5,
                 show_info: bool = False,
                 ):

        self.seed = seed
        self.n_splits = n_splits
        self.show_info = show_info
        self.train_data_read_path = "../input/tabular-playground-series-aug-2022/train.csv"
        self.test_data_read_path = "../input/tabular-playground-series-aug-2022/test.csv"
        self.ss_data_read_path = "../input/tabular-playground-series-aug-2022/sample_submission.csv"
        self.model_save_path = "models"
        if not os.path.exists(self.model_save_path): os.makedirs(self.model_save_path)

cfg = CFG(seed=42, show_info=True)

### **Helpers**

In [3]:
def breaker(num: int=50, char: str="*") -> None:
    print("\n" + num*char + "\n")


def preprocess(df) -> None:
    df.attribute_0 = df.attribute_0.map({"material_5" : 0, "material_7" : 1}) 
    df.attribute_1 = df.attribute_1.map({"material_5" : 0, "material_7" : 1, "material_6" : 2}) 
    

def print_scores(accuracy: float, auc: float, precision: np.ndarray, recall: np.ndarray, f_score: np.ndarray) -> None:
    print(f"Accuracy  : {accuracy:.5f}")
    print(f"ROC-AUC   : {auc:.5f}")
    print(f"Precision : {precision}")
    print(f"Recall    : {recall}")
    print(f"F-Score   : {f_score}")
    

def get_scores(y_true: np.ndarray, y_pred: np.ndarray) -> tuple:
    accuracy = accuracy_score(y_pred, y_true)
    try:
        auc = roc_auc_score(y_pred, y_true)
    except:
        auc = 0.0
    precision, recall, f_score, _ = precision_recall_fscore_support(y_pred, y_true)

    return accuracy, auc, precision, recall, f_score

### **Model**

In [4]:
class Model(object):
    def __init__(self, model_name: str, preprocessor, seed: int):
        self.model_name = model_name

        if self.model_name == "lgr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", LogisticRegression(random_state=seed)),
                ]
            )
        
        elif self.model_name == "knc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", KNeighborsClassifier()),
                ]
            )

        
        elif self.model_name == "dtc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", DecisionTreeClassifier(random_state=seed)),
                ]
            )

        elif self.model_name == "etc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", ExtraTreeClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "rfc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", RandomForestClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "gbc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", GradientBoostingClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "abc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", AdaBoostClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "etcs":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", ExtraTreesClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "gnb":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", GaussianNB()),
                ]
            )
        
        elif self.model_name == "xgbc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", XGBClassifier(random_state=seed)),
                ]
            )

### **Train**

In [5]:
names = ["lgr", "knc", "gnb", "dtc", "etc", "abc", "gbc", "etcs", "rfc", "xgbc"]

df = pd.read_csv(cfg.train_data_read_path)
df = df.drop(columns=["id", "product_code"])
preprocess(df)

if cfg.show_info:
    breaker()
    for val in set(df.failure):
        print(f"Class {val} count : {df[df.failure == val].shape[0]}")

X = df.iloc[:, :-1].copy().values
y = df.iloc[:, -1].copy().values

features = [i for i in range(X.shape[1])]

feature_transformer = Pipeline(
    steps=[
        ("Simple_Imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
        ("Standard_Scaler", StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("features", feature_transformer, features),
    ]
)

best_auc = 0.0
for name in names:
    fold = 1
    if cfg.show_info: breaker()
    for tr_idx, va_idx in KFold(n_splits=cfg.n_splits, random_state=cfg.seed, shuffle=True).split(X):
        X_train, X_valid, y_train, y_valid = X[tr_idx], X[va_idx], y[tr_idx], y[va_idx]
        my_pipeline = Model(name, preprocessor, cfg.seed)
        my_pipeline.model.fit(X_train, y_train)

        y_pred = my_pipeline.model.predict(X_valid)
        if cfg.show_info:
            acc, auc, pre, rec, f1 = get_scores(y_valid, y_pred)
            print(f"{my_pipeline.model_name}, {fold}\n")
            print_scores(acc, auc, pre, rec, f1)
            print("")
        else:
            _, auc, _, _, _ = get_scores(y_valid, y_pred)

        if auc > best_auc:
            best_auc = auc
            model_fold_name = f"{name}_{fold}"
            
            with open(os.path.join(cfg.model_save_path, f"best_model.pkl"), "wb") as fp:
                pickle.dump(my_pipeline.model, fp)
        fold += 1
    

if cfg.show_info: 
    breaker()
    print(f"Best Model : {model_fold_name.split('_')[0]}, Best Fold : {model_fold_name.split('_')[1]}")

breaker()


**************************************************

Class 0 count : 20921
Class 1 count : 5649

**************************************************

lgr, 1

Accuracy  : 0.80072
ROC-AUC   : 0.61513
Precision : [0.99812074 0.00567644]
Recall    : [0.80169811 0.42857143]
F-Score   : [0.88919117 0.01120448]

lgr, 2

Accuracy  : 0.78265
ROC-AUC   : 0.59150
Precision : [0.99927885 0.0017331 ]
Recall    : [0.78300998 0.4       ]
F-Score   : [0.87802302 0.00345125]

lgr, 3

Accuracy  : 0.78303
ROC-AUC   : 0.59169
Precision : [0.99927919 0.00173611]
Recall    : [0.7833867 0.4      ]
F-Score   : [0.87825995 0.00345722]

lgr, 4

Accuracy  : 0.79131
ROC-AUC   : 0.69574
Precision : [0.99952426 0.0027027 ]
Recall    : [0.79148616 0.6       ]
F-Score   : [0.88342268 0.00538117]

lgr, 5

Accuracy  : 0.77832
ROC-AUC   : 0.58952
Precision : [0.99855002 0.00340136]
Recall    : [0.77903469 0.4       ]
F-Score   : [0.8752383  0.00674536]


**************************************************

knc, 1

Accurac

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


etcs, 2

Accuracy  : 0.78284
ROC-AUC   : 0.00000
Precision : [1. 0.]
Recall    : [0.78283779 0.        ]
F-Score   : [0.87819295 0.        ]

etcs, 3

Accuracy  : 0.78303
ROC-AUC   : 0.55831
Precision : [9.99519462e-01 8.68055556e-04]
Recall    : [0.78327998 0.33333333]
F-Score   : [0.87828565 0.0017316 ]

etcs, 4

Accuracy  : 0.79093
ROC-AUC   : 0.39554
Precision : [0.99976213 0.        ]
Recall    : [0.79107849 0.        ]
F-Score   : [0.88326153 0.        ]

etcs, 5

Accuracy  : 0.77813
ROC-AUC   : 0.38929
Precision : [0.99927501 0.        ]
Recall    : [0.77857277 0.        ]
F-Score   : [0.87522489 0.        ]


**************************************************

rfc, 1

Accuracy  : 0.80015
ROC-AUC   : 0.47194
Precision : [9.98590557e-01 9.46073794e-04]
Recall    : [0.80101752 0.14285714]
F-Score   : [0.88895859 0.0018797 ]

rfc, 2

Accuracy  : 0.78152
ROC-AUC   : 0.39128
Precision : [0.99831731 0.        ]
Recall    : [0.78255135 0.        ]
F-Score   : [0.87736347 0.        ]

r

### **Predict**

In [6]:
df = pd.read_csv(cfg.test_data_read_path)
df = df.drop(columns=["id", "product_code"])
preprocess(df)

X = df.copy().values

model = pickle.load(open(os.path.join(cfg.model_save_path, f"best_model.pkl"), "rb"))
y_pred = model.predict_proba(X)[:, 1]

ss_df = pd.read_csv(cfg.ss_data_read_path)
ss_df["failure"] = y_pred
ss_df.to_csv("submission.csv", index=False)